In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error,classification_report, roc_curve 

import re
from functools import reduce
from collections import Counter
from imputation import find_baths, find_bedrooms, find_beds, data_imputation,hrt_pred, reduce_groups,get_n_1_dummies,get_n_dummies

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
%matplotlib inline

In [14]:
model_data=pd.read_csv('modeldata.csv')
model_data.head()

,Unnamed: 0,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_since_years,Success_rate,mean_price,host_in_sfo,licensed,top_amenities_count,host_verification_count
0,0,0,100,98,t,1.0,t,t,Group1,Entire serviced apartment,Entire home/apt,3,bath group1,1.0,2.0,2,30,2.0,2.0,1125.0,1125.0,2.0,1125.0,315,42,0,4.87,4.95,4.95,4.90,4.98,f,1,1,0,0,2.08,13.4,0.71303,172.2,1.0,0.0,3,5
1,1,2,100,71,f,2.0,t,t,Group1,Entire rental unit,Entire home/apt,5,bath group1,2.0,3.0,30,60,30.0,30.0,60.0,60.0,30.0,60.0,111,0,0,4.88,4.87,4.89,4.85,4.77,f,1,1,0,0,0.72,12.9,0.00000,235.0,0.0,0.0,2,5
2,2,0,100,100,f,10.0,t,t,Group1,Private room in rental unit,Private room,2,bath group3,1.0,1.0,32,60,32.0,32.0,60.0,60.0,32.0,60.0,19,0,0,4.20,3.87,4.67,4.60,4.73,f,9,0,9,0,0.13,12.6,0.00000,62.1,1.0,0.0,1,5
3,3,0,100,100,f,10.0,t,t,Group1,Private room in rental unit,Private room,2,bath group3,1.0,1.0,32,90,32.0,32.0,90.0,90.0,32.0,90.0,8,0,0,4.63,4.38,4.75,4.75,4.63,f,9,0,9,0,0.09,12.6,0.00000,62.1,1.0,0.0,1,5
4,4,1,94,0,f,2.0,t,t,Group1,Entire condominium (condo),Entire home/apt,4,bath group1,2.0,2.0,7,111,7.0,7.0,111.0,111.0,7.0,111.0,28,0,0,4.87,5.00,4.94,5.00,4.94,f,2,2,0,0,0.19,12.5,0.00000,895.0,1.0,1.0,2,4


In [20]:
encoded_data=get_n_1_dummies()
encoded_data.shape

(6340, 64)

In [5]:
encoded_data.shape

(6340, 64)

# KNN Regression

In [107]:
X=encoded_data.drop(['Success_rate'],axis=1)
Y=round(encoded_data['Success_rate'],4)

In [108]:
#train, valid & test test split
X_train,X_valid,Y_train,Y_valid=train_test_split(X,Y,test_size=0.1,random_state=1)
X_train,X_test,Y_train,Y_test=train_test_split(X_train,Y_train,test_size=0.1111,random_state=1)
X_train.shape,X_valid.shape,X_test.shape

#normalize the predictors
cols=X_train.columns
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_valid=scaler.transform(X_valid)
X_test=scaler.transform(X_test)
X_train=pd.DataFrame(X_train,columns=cols)
X_valid=pd.DataFrame(X_valid,columns=cols)
X_test=pd.DataFrame(X_test,columns=cols)

In [89]:
# to check balance of dataset split
Y1=[1 for i in Y_test.values.tolist() if i>0.5 ]
#Counter(Y1).keys(), Counter(Y1).values()


In [109]:
# find best k
k=np.arange(1,20,1).tolist()
for i in k:
    knn=KNeighborsRegressor(n_neighbors=i)
    knn.fit(X_train.values,Y_train.values)
    Y_pred_valid=knn.predict(X_valid.values)
    print('Root Mean squared error: {0:0.4f}'.format(mean_squared_error(Y_valid,Y_pred_valid,squared=False)), 'for k=',i)

Root Mean squared error: 0.3861 for k= 1
Root Mean squared error: 0.3443 for k= 2
Root Mean squared error: 0.3392 for k= 3
Root Mean squared error: 0.3274 for k= 4
Root Mean squared error: 0.3256 for k= 5
Root Mean squared error: 0.3204 for k= 6
Root Mean squared error: 0.3186 for k= 7
Root Mean squared error: 0.3159 for k= 8
Root Mean squared error: 0.3163 for k= 9
Root Mean squared error: 0.3180 for k= 10
Root Mean squared error: 0.3175 for k= 11
Root Mean squared error: 0.3199 for k= 12
Root Mean squared error: 0.3198 for k= 13
Root Mean squared error: 0.3196 for k= 14
Root Mean squared error: 0.3187 for k= 15
Root Mean squared error: 0.3191 for k= 16
Root Mean squared error: 0.3202 for k= 17
Root Mean squared error: 0.3204 for k= 18
Root Mean squared error: 0.3197 for k= 19


In [91]:
#k=17 gives better accuracy in validation. Let's use it to predict test data
knn=KNeighborsRegressor(n_neighbors=8)
knn.fit(X_train.values,Y_train.values)
y_pred_train=knn.predict(X_train.values)
y_pred_valid=knn.predict(X_valid.values)
y_pred_test=knn.predict(X_test.values)
#accuracy
print('Training RMSE:',mean_squared_error(Y_train,y_pred_train,squared=False))
print('Validation RMSE:',mean_squared_error(Y_valid,y_pred_valid, squared=False))
print('Test RMSE:',mean_squared_error(Y_test,y_pred_test, squared=False))

Training RMSE: 0.2724020238364067
Validation RMSE: 0.31594729753995626
Test RMSE: 0.32542388790552623


In [110]:
df=pd.DataFrame({'Y_test_actual':Y_test,'Y_prediction':y_pred_test})
df

,Y_test_actual,Y_prediction
2241,0.5773,0.592938
166,0.7000,0.502887
5770,0.2759,0.767737
3013,0.4511,0.439487
3160,0.2167,0.666387
3735,0.0500,0.304275
5487,0.8000,0.651338
1337,0.2083,0.549063
5452,0.6015,0.384100
4386,0.0000,0.000000


# KNN Classification

In [21]:
for i in encoded_data.index:
    if encoded_data.loc[i,'Success_rate']>0.50:
        encoded_data.loc[i,'Successful']=1
    else:
        encoded_data.loc[i,'Successful']=0
encoded_data['Successful'].value_counts()
encoded_data.drop(['Success_rate'],axis=1,inplace=True)

In [22]:
X=encoded_data.drop(['Successful'],axis=1)
Y=encoded_data['Successful']

#train, valid & test test split
X_train,X_valid,Y_train,Y_valid=train_test_split(X,Y,test_size=0.1,random_state=1)
X_train,X_test,Y_train,Y_test=train_test_split(X_train,Y_train,test_size=0.1111,random_state=1)
X_train.shape,X_valid.shape,X_test.shape

#normalize the predictors
cols=X_train.columns
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_valid=scaler.transform(X_valid)
X_test=scaler.transform(X_test)
X_train=pd.DataFrame(X_train,columns=cols)
X_valid=pd.DataFrame(X_valid,columns=cols)
X_test=pd.DataFrame(X_test,columns=cols)

In [23]:
# find best k
k=np.arange(1,20,1).tolist()
for i in k:
    knn=KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train.values,Y_train.values)
    Y_pred_valid=knn.predict(X_valid.values)
    print('Model accuracy score: {0:0.4f}'. format(accuracy_score(Y_valid, Y_pred_valid)), 'for k=',i)
    print(classification_report(Y_valid, Y_pred_valid))

Model accuracy score: 0.6562 for k= 1
              precision    recall  f1-score   support

         0.0       0.64      0.67      0.66       311
         1.0       0.67      0.64      0.66       323

    accuracy                           0.66       634
   macro avg       0.66      0.66      0.66       634
weighted avg       0.66      0.66      0.66       634

Model accuracy score: 0.6246 for k= 2
              precision    recall  f1-score   support

         0.0       0.58      0.83      0.69       311
         1.0       0.72      0.42      0.54       323

    accuracy                           0.62       634
   macro avg       0.65      0.63      0.61       634
weighted avg       0.65      0.62      0.61       634

Model accuracy score: 0.6451 for k= 3
              precision    recall  f1-score   support

         0.0       0.63      0.66      0.64       311
         1.0       0.66      0.63      0.65       323

    accuracy                           0.65       634
   macro avg  

In [24]:
#k=2,13 has better accuracy(when cutoff is 0.85)in validation data. let's use it to find the accuracy in test data
#k=7 has better accuracy in validation data(overalll accuracy: 0.6672 when cutoff is 0.5)
knn=KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train.values, Y_train.values)
Y_pred_test=knn.predict(X_test.values)
print('Model accuracy score in test set:{0:0.4f}'.format(accuracy_score(Y_test,Y_pred_test)))
print(classification_report(Y_test, Y_pred_test))

Model accuracy score in test set:0.6151
              precision    recall  f1-score   support

         0.0       0.64      0.63      0.63       336
         1.0       0.59      0.60      0.59       298

    accuracy                           0.62       634
   macro avg       0.61      0.61      0.61       634
weighted avg       0.62      0.62      0.62       634

